In [1]:
# libraries for general use
import numpy as np
import pandas as pd

# reading our dataset in a pd.DataFrame object
df = pd.read_csv('originOfMusic.csv')
df.rename(columns={'V117': 'lat', 'V118': 'long'}, inplace=True)
# structure of the dataset:
# print(df.head(2))

# we will also need countries names and codes for visualization
excel = pd.read_excel('countries.xlsx', index_col=1, header=None)
# print(excel.head())

# counting how many track per country are
tracks_per_country = df.groupby('long').describe()['V1']
tracks_per_country = tracks_per_country['count']

new_columns = pd.DataFrame(tracks_per_country.sort_values(ascending=False))

new_columns['country_name'] = [excel.iloc[i][0] for i in range(0, len(excel))]
new_columns['country_label'] = np.arange(0, 33)
new_columns['country_code'] = [excel.iloc[i][2] for i in range(0, len(excel))]

print(new_columns.head(2))

# adding to the dataframe 2 new columns:
# labels are used as target for our model
# split_l is used for cross-validation (StratifiedSuffleSplit and StratifiedKFold need a criterion for preserving
# the percentage of sample for each class)

df['labels'] = [tuple([lat, long]) for lat, long in zip(df['lat'], df['long'])]
df['split_l'] = [new_columns.loc[long]['country_label'] for long in df['long'].values]

print(df['labels'])

# a priori visualization
import plotly.graph_objects as go

fig = go.Figure(data=go.Choropleth(
    locations=new_columns['country_code'],
    z=new_columns['count'],
    text=new_columns['country_name'],
    colorscale='Blues',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
))

fig.show()

# the loss function
from geopy.distance import geodesic
from shapely.geometry import Point

from sklearn.metrics.scorer import make_scorer


def score_function(true_labels, prediction):
    s = []
    for pred, true in zip(prediction, true_labels):
        s.append(geodesic(pred, true))

    return abs(float(str(np.mean(s))[:-3]))


score_func = make_scorer(score_function, greater_is_better=False)

       count country_name  country_label country_code
long                                                 
 77.2   69.0        India              0          IND
-8.0    66.0         Mali              1          MLI


In [3]:
# from sklearn import preprocessing
# normed_data1 = preprocessing.robust_scale(df.iloc[:, :-4])
# normed_data = preprocessing.normalize(df.iloc[:, :-4])


In [5]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(df.iloc[:, :-4], df.iloc[:]['labels'],
                                                                    test_size=0.2, random_state=42)

In [6]:
strKfold = model_selection.StratifiedKFold(n_splits=10, random_state=0)
strShuSplit = model_selection.StratifiedShuffleSplit(n_splits=10, test_size=.20, random_state=0)
kFold = model_selection.KFold(n_splits=10, random_state=0, shuffle=True)

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

param = {'activation':['relu', 'tanh'], 
         'alpha':[0.0001, 0.001, 0.00001, 0.01], 
         'batch_size':[100, 200, 128], 
         'tol':[0.0001, 0.001, 0.01, 0.00001], 
         'momentum':[0.9, 0.8, 0.7], 
        }

model = MLPRegressor(hidden_layer_sizes=(100, 50, 50), solver='lbfgs', random_state=0, 
                     warm_start=True,
                     early_stopping=True, learning_rate='adaptive', 
                     learning_rate_init=0.001, max_iter=200, 
                     shuffle=True, validation_fraction=0.1,
                     n_iter_no_change=10)

clf = GridSearchCV(model, param, verbose=10, cv=kFold)

clf.fit(X_train, list(y_train))


Fitting 10 folds for each of 288 candidates, totalling 2880 fits
[CV] activation=relu, alpha=0.0001, batch_size=100, momentum=0.9, tol=0.0001 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:420: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.6s remaining:    0.0s


[CV]  activation=relu, alpha=0.0001, batch_size=100, momentum=0.9, tol=0.0001, score=0.031, total=   2.6s
[CV] activation=relu, alpha=0.0001, batch_size=100, momentum=0.9, tol=0.0001 


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:420: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.1s remaining:    0.0s


[CV]  activation=relu, alpha=0.0001, batch_size=100, momentum=0.9, tol=0.0001, score=0.049, total=   3.5s
[CV] activation=relu, alpha=0.0001, batch_size=100, momentum=0.9, tol=0.0001 


...

[CV]  activation=relu, alpha=0.0001, batch_size=100, momentum=0.7, tol=0.0001, score=0.037, total=   2.1s
[CV] activation=relu, alpha=0.0001, batch_size=100, momentum=0.7, tol=0.001 


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:420: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



[CV]  activation=relu, alpha=0.0001, batch_size=100, momentum=0.7, tol=0.001, score=0.031, total=   2.0s
[CV] activation=relu, alpha=0.0001, batch_size=100, momentum=0.7, tol=0.001 


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:420: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



[CV]  activation=relu, alpha=0.0001, batch_size=100, momentum=0.7, tol=0.001, score=0.049, total=   3.4s
[CV] activation=relu, alpha=0.0001, batch_size=100, momentum=0.7, tol=0.001 


In [ ]:
print(clf.best_params_)
print(clf.best_estimator_)

In [ ]:
# mean over all distances between prediction and true labels
pred = pd.DataFrame(clf.best_estimator_.predict( X_test))
s = score_function(prediction=pred, true_labels=y_test)
print('mean_distance: ',s)
